In [8]:
# Import libraries
from flair.data import Sentence
from flair.models import SequenceTagger
from cassis import *

In [11]:
tsfile = 'ts1.xml'
csfile = './output1.xmi/document.txt.xmi'

TOKEN_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token"
SENTENCE_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"

# Load Typesystem
with open(tsfile, 'rb') as f:
    typesystem = load_typesystem(f)

# Load Cas
with open(csfile, 'rb') as f:
    cas = load_cas_from_xmi(f, typesystem=typesystem)

In [12]:
# Get annotation type
AnnotationType = typesystem.get_type(TOKEN_TYPE)

In [13]:
tokens = list(cas.select(TOKEN_TYPE))
words = [cas.get_covered_text(token) for token in tokens]
# tokens

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: get_covered_text is deprecated. Use annotation.get_covered_text()
  


In [14]:
sentence = ''
for word in words:
    sentence += word + ' '

sentence = Sentence(sentence)
model_name = 'ner'
nlp = SequenceTagger.load(model_name)

nlp.predict(sentence)

2020-06-15 15:25:15,345 loading file /home/allan/snap/jupyter/6/.flair/models/en-ner-conll03-v0.4.pt


[Sentence: "Courtney de Souza has been living with her daughter Susan in Berlin for about 6 years ."   [− Tokens: 17  − Token-Labels: "Courtney <B-PER> de <I-PER> Souza <E-PER> has been living with her daughter Susan <S-PER> in Berlin <S-LOC> for about 6 years ."]]

### #testing start

In [15]:
NERType = typesystem.create_type("webanno.custom.flair-ner")
typesystem.add_feature(type_=NERType, name='ner', rangeTypeName='uima.cas.String')

In [16]:
# begin = 0
for sent in range(len(sentence)):
    token = sentence[sent]
    word = str(token).split()
    tag = sentence[sent].get_tag('ner')
#     end = begin + len(word)
    cas_sentence = AnnotationType(begin = tokens[sent].begin, 
                                  end = tokens[sent].end)
    ner_annotation = NERType(begin = tokens[sent].begin,
                             end = tokens[sent].end,
                             ner = tag.value)
    cas.add_annotation(cas_sentence)
    cas.add_annotation(ner_annotation)
#     begin = end + 1
xmi = cas.to_xmi('./output2.xmi', pretty_print=True)

### #testing end